# Smart Bot Orchestration - Automated Grid Trading

## 🎯 Overview
Intelligent bot management system that analyzes signals, monitors performance, and automatically deploys/manages grid trading strategies on Hummingbot.

## ✨ Key Features
- **Signal-Based Trading**: Automatically deploys controllers based on EMA trend signals
- **Performance Monitoring**: Tracks P&L, volume, and runtime for each controller
- **Intelligent Management**: Stops underperforming or conflicting controllers
- **Multi-Bot Support**: Manages up to 3 bot instances simultaneously
- **Auto-Archive**: Automatically archives bots when all controllers are stopped
- **Telegram Notifications**: Real-time updates at every step

## 📋 How It Works

### Step 1: Signal Analysis
- Fetches top 5 long and top 5 short signals from MongoDB
- Uses EMA trend feature with configurable intensity thresholds
- Sends signal summary via Telegram

### Step 2: Check All Active Bots
- Searches for ALL bots matching pattern `trend_follower_grid*`
- Displays controllers, P&L, and runtime for each bot
- Runtime calculated individually per bot deployment time
- Sends comprehensive status via Telegram

### Step 3: Analyze & Stop Controllers
- **Stops controllers if:**
  - Runtime > min hours AND P&L < threshold (-0.1% default)
  - Signal direction changed (long→short or short→long)
- **Keeps controllers if:**
  - Runtime < min hours (protection period)
  - Performing well and signal still valid
- **Inactive controllers** (already stopped) don't count toward capacity

### Step 4: Archive Dead Bots
- Identifies bots where ALL controllers are stopped
- Archives these bots to free up slots
- Max 3 active bots allowed simultaneously

### Step 5: Deploy New Controllers
- Checks bot limit (max 3 active bots)
- Identifies opportunities not already covered
- Skips recently stopped markets (no immediate redeployment)
- Creates grid configs and deploys new bot instance
- Sends detailed deployment report via Telegram

### Step 6: Final Summary
- Complete orchestration report via Telegram
- Shows stopped, deployed, kept, and archived bots
- Management rules applied across all bots

## ⚙️ Configuration

| Setting | Default | Description |
|---------|---------|-------------|
| `bot_name` | `trend_follower_grid` | Bot name pattern (matches all with this prefix) |
| `max_controllers_per_instance` | 10 | Maximum total controllers across all bots |
| `max_active_bots` | 3 | Maximum number of bot instances allowed |
| `min_runtime_hours` | 4 | Protection period before stopping (per controller) |
| `min_pnl_pct_to_keep` | -0.1% | P&L threshold to keep running |
| `stop_on_opposing_signal` | True | Stop if signal flips direction |
| `total_amount_quote` | $400 | Capital per grid |
| `leverage` | 20x | Position leverage |
| `take_profit` | 0.16% | Grid take profit |

## 🔄 Typical Workflow

### First Run (No Active Bots)
```
1. Load signals → Telegram notification
2. No bots found → Deploy fresh
3. Create 10 controllers (5 long + 5 short)
4. Deploy bot1 → Telegram notification
5. Final summary → Telegram notification
```

### Multi-Bot Scenario with Archiving
```
1. Load signals → Telegram notification
2. Find 3 active bots:
   - bot1 (runtime: 8h, 3 active, 7 stopped)
   - bot2 (runtime: 6h, 7 active, 0 stopped)
   - bot3 (runtime: 2h, 0 active, 10 stopped - all dead)
3. Analyze 20 controllers across all bots
4. Archive bot3 (all controllers stopped)
5. Bot limit: 2/3 active (space for 1 more)
6. Deploy bot4 with 3 new controllers
7. Final summary → Telegram notification
```

## 📱 Telegram Notifications

You'll receive **3-4 notifications** per run:

1. **🎯 Trading Signals** - Top opportunities identified
2. **🤖 Current Bot Status** - All active bots with P&L breakdown (if bots exist)
3. **🚀 Bot Deployed** - New deployment details (if deployed)
4. **🎉 Orchestration Complete** - Final summary with all actions

## 🚀 Quick Start

1. **Set up environment**:
   - Ensure Hummingbot API is running on localhost:8000
   - Configure Telegram bot token in .env
   - Run feature engineering notebook first

2. **Run cells in order**:
   - Each cell is designed to run sequentially
   - Telegram notifications happen automatically

3. **Check results**:
   - Console output shows detailed progress
   - Telegram messages provide mobile-friendly summaries
   - Hummingbot dashboard shows active trading

## ⚠️ Important Notes

- **Bot Naming**: Pattern `trend_follower_grid` matches all deployments (e.g., `trend_follower_grid-20251014-135851`)
- **Multi-Bot**: System handles up to 3 bot instances automatically
- **Auto-Archive**: Bots with all controllers stopped are automatically archived
- **Runtime Tracking**: Each controller tracks its own bot's deployment time
- **Controller Limit**: Max 10 total ACTIVE controllers across ALL bots
- **Protection Period**: Controllers safe from stopping for first 4 hours
- **Stopped Controllers**: Don't count toward the 10 controller limit

## 🛠️ Troubleshooting

**No signals found?**
- Run feature engineering notebook first
- Check MongoDB connection
- Verify signal intensity thresholds

**Bot won't deploy?**
- Check if max bot limit (3) is reached
- Verify at least one bot slot available
- Check Hummingbot API is running
- Ensure exchange API keys are configured

**Too many controllers stopped?**
- Increase `min_runtime_hours` for longer protection
- Adjust `min_pnl_pct_to_keep` to be more lenient
- Disable `stop_on_opposing_signal` if signals fluctuate

**Bots not archiving?**
- System only archives when ALL controllers are stopped
- Check `auto_stop_controllers` is enabled
- Manually stop bot via Hummingbot if needed

## 📊 Performance Tracking

Monitor your bots' performance through:
- **Telegram**: Real-time notifications with multi-bot P&L updates
- **Notebook Output**: Detailed breakdown by bot and controller
- **Hummingbot Dashboard**: Live trading view
- **MongoDB**: Historical feature and signal data

In [ ]:
# Imports and Setup
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
load_dotenv()

from hummingbot_api_client import HummingbotAPIClient
from core.features import FeatureStorage
from core.notifiers import NotificationManager, NotificationMessage

# Initialize
storage = FeatureStorage()
notification_manager = NotificationManager()

print("✅ Smart Bot Orchestration initialized")
print(f"🔔 Enabled notifiers: {', '.join(notification_manager.get_enabled_notifiers())}")

In [ ]:
# Configuration
CONFIG = {
    # Bot instance settings - SIMPLIFIED: bot name = strategy name
    'bot_name': 'trend_follower_grid',  # This is now the actual bot name
    'credentials_profile': 'master_account',
    'connector_name': 'binance_perpetual',
    'hummingbot_host': 'localhost',
    
    # Signal settings
    'feature_name': 'ema_trend',
    'signal_category': 'tf',
    'min_long_intensity': 0.7,
    'min_short_intensity': -0.7,
    'top_n_per_side': 5,
    
    # Management rules
    'max_controllers_per_instance': 10,
    'max_active_bots': 3,  # Maximum number of bot instances allowed
    'min_runtime_hours': 0.05,
    'stop_on_opposing_signal': True,
    'min_pnl_pct_to_keep': -3.0,
    
    # Grid parameters
    'total_amount_quote': '400',
    'leverage': 20,
    'position_mode': 'HEDGE',
    'max_open_orders': 2,
    'max_orders_per_batch': 1,
    'min_order_amount_quote': '6',
    'activation_bounds': '0.004',
    'min_spread_between_orders': '0.002',
    'take_profit': '0.0016',
    
    # Grid range multipliers
    'grid_range_multiplier': {
        'long': {'start_offset': -0.25, 'end_offset': 0.75, 'limit_offset': -0.35},
        'short': {'start_offset': -0.75, 'end_offset': 0.25, 'limit_offset': 0.35}
    },
    
    # Execution settings
    'auto_stop_controllers': True,
    'auto_deploy': True,
    'send_telegram_notifications': True,  # Send notifications at each step
}

print("📊 Configuration loaded:")
print(f"  Bot name: {CONFIG['bot_name']}")
print(f"  Max controllers: {CONFIG['max_controllers_per_instance']}")
print(f"  Max active bots: {CONFIG['max_active_bots']}")
print(f"  Min runtime: {CONFIG['min_runtime_hours']} hours")
print(f"  Min P&L to keep: {CONFIG['min_pnl_pct_to_keep']}%")
print(f"  Telegram notifications: {CONFIG['send_telegram_notifications']}")

In [ ]:
# Connect to MongoDB
await storage.connect()
print("✅ Connected to MongoDB feature storage")

## 1. Load Current Signals

In [ ]:
# Fetch top signals
long_signals = await storage.get_signals(
    category=CONFIG['signal_category'],
    min_value=CONFIG['min_long_intensity'],
    limit=CONFIG['top_n_per_side'] * 2
)

short_signals = await storage.get_signals(
    category=CONFIG['signal_category'],
    max_value=CONFIG['min_short_intensity'],
    limit=CONFIG['top_n_per_side'] * 2
)

# Create signal lookup dict
signal_map = {}
for sig in long_signals:
    signal_map[sig.trading_pair] = {'value': sig.value, 'direction': 'long'}
for sig in short_signals:
    signal_map[sig.trading_pair] = {'value': sig.value, 'direction': 'short'}

print(f"🟢 Found {len(long_signals)} long signals (value > {CONFIG['min_long_intensity']})")
print(f"🔴 Found {len(short_signals)} short signals (value < {CONFIG['min_short_intensity']})")

if long_signals:
    print("\n🟢 Top Long Signals:")
    for i, sig in enumerate(long_signals[:CONFIG['top_n_per_side']], 1):
        print(f"  {i}. {sig.trading_pair:12s} | Intensity: {sig.value:+.3f}")

if short_signals:
    print("\n🔴 Top Short Signals:")
    for i, sig in enumerate(short_signals[:CONFIG['top_n_per_side']], 1):
        print(f"  {i}. {sig.trading_pair:12s} | Intensity: {sig.value:+.3f}")

## 2. Fetch Active Bots and Controllers

In [ ]:
# Fetch active bot status - MULTI-BOT SUPPORT
bot_controllers = []
running_bot_names = []  # Track ALL matching bot names
bot_deployed_times = {}  # Map bot_name -> deployed_at

print("🤖 Fetching active bot status from Hummingbot API...\n")

async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
    try:
        # Check if our bots are running
        active_bots_data = await client.bot_orchestration.get_active_bots_status()
        
        if active_bots_data and active_bots_data.get('status') == 'success':
            bots_data = active_bots_data.get('data', {})
            
            # Find ALL bots that match our bot_name pattern
            for bot_name, bot_info in bots_data.items():
                if CONFIG['bot_name'] in bot_name:
                    running_bot_names.append(bot_name)
                    
                    print(f"✅ Found active bot: {bot_name}")

                    # Get deployment time for THIS specific bot
                    bot_runs = await client.bot_orchestration.get_bot_runs(
                        bot_name=bot_name,
                        run_status="CREATED"
                    )

                    bot_deployed_at = None
                    if bot_runs.get('status') == 'success' and bot_runs.get('data'):
                        deployed_at_str = bot_runs['data'][0].get('deployed_at')
                        if deployed_at_str:
                            bot_deployed_at = datetime.fromisoformat(deployed_at_str.replace('Z', '+00:00'))
                            bot_deployed_times[bot_name] = bot_deployed_at
                            print(f"   Deployed: {bot_deployed_at.strftime('%Y-%m-%d %H:%M UTC')}")

                    # Get controller configs for THIS bot
                    controller_configs_list = await client.controllers.get_bot_controller_configs(
                        bot_name=bot_name
                    )

                    # Convert to dict
                    controller_configs = {}
                    if isinstance(controller_configs_list, list):
                        for config in controller_configs_list:
                            if isinstance(config, dict) and 'id' in config:
                                controller_configs[config['id']] = config

                    # Parse controllers for THIS bot
                    performance = bot_info.get('performance', {})

                    for ctrl_id, ctrl_perf in performance.items():
                        if isinstance(ctrl_perf, dict) and 'performance' in ctrl_perf:
                            perf_data = ctrl_perf['performance']
                            ctrl_config = controller_configs.get(ctrl_id, {})

                            trading_pair = ctrl_config.get('trading_pair', 'UNKNOWN')
                            side = ctrl_config.get('side', 0)
                            direction = 'long' if side == 1 else 'short' if side == 2 else 'unknown'

                            # Calculate runtime for THIS controller based on ITS bot's deployed_at
                            runtime_hours = 0
                            if bot_deployed_at:
                                runtime = datetime.now(timezone.utc) - bot_deployed_at
                                runtime_hours = runtime.total_seconds() / 3600

                            bot_controllers.append({
                                'controller_id': ctrl_id,
                                'trading_pair': trading_pair,
                                'direction': direction,
                                'status': ctrl_perf.get('status', 'unknown'),
                                'pnl_quote': perf_data.get('global_pnl_quote', 0),
                                'pnl_pct': perf_data.get('global_pnl_pct', 0),
                                'volume': perf_data.get('volume_traded', 0),
                                'runtime_hours': runtime_hours,
                                'manual_kill_switch': ctrl_config.get('manual_kill_switch', False),
                                'bot_name': bot_name,  # NEW: Track which bot this controller belongs to
                                'config': ctrl_config
                            })
            
            if not running_bot_names:
                print(f"ℹ️  No active bots found with name pattern '{CONFIG['bot_name']}'")
                print("   Will deploy fresh instance")

    except Exception as e:
        print(f"❌ Error fetching bot status: {e}")
        import traceback
        traceback.print_exc()

# Display bot status - UPDATED FOR MULTI-BOT
if bot_controllers:
    total_pnl = sum(ctrl['pnl_quote'] for ctrl in bot_controllers)
    total_volume = sum(ctrl['volume'] for ctrl in bot_controllers)
    winners = [c for c in bot_controllers if c['pnl_quote'] > 0]
    losers = [c for c in bot_controllers if c['pnl_quote'] < 0]
    
    print("\n" + "=" * 80)
    print(f"🤖 ACTIVE BOTS: {len(running_bot_names)} bot(s)")
    print("=" * 80)
    print(f"Total Controllers: {len(bot_controllers)} | Winners: {len(winners)} | Losers: {len(losers)}")
    print(f"Total P&L: ${total_pnl:+.2f} | Volume: ${total_volume:,.0f}")
    
    # Group controllers by bot
    controllers_by_bot = {}
    for ctrl in bot_controllers:
        bot = ctrl['bot_name']
        if bot not in controllers_by_bot:
            controllers_by_bot[bot] = []
        controllers_by_bot[bot].append(ctrl)
    
    # Display each bot's controllers
    for bot_name, controllers in controllers_by_bot.items():
        bot_pnl = sum(c['pnl_quote'] for c in controllers)
        bot_runtime = controllers[0]['runtime_hours'] if controllers else 0
        bot_winners = [c for c in controllers if c['pnl_quote'] > 0]
        bot_losers = [c for c in controllers if c['pnl_quote'] < 0]
        
        print(f"\n{'─' * 80}")
        print(f"📦 Bot: {bot_name}")
        print(f"   Runtime: {bot_runtime:.1f}h | Controllers: {len(controllers)} | P&L: ${bot_pnl:+.2f}")
        print(f"   Winners: {len(bot_winners)} | Losers: {len(bot_losers)}")
    
    if winners:
        print(f"\n🎯 ALL WINNERS ({len(winners)}):")
        for ctrl in sorted(winners, key=lambda x: x['pnl_quote'], reverse=True):
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            print(f"  {dir_emoji} {ctrl['trading_pair']:12s} {ctrl['direction']:5s} | "
                  f"${ctrl['pnl_quote']:+7.2f} ({ctrl['pnl_pct']:+7.2f}%) | "
                  f"Vol: ${ctrl['volume']:>8,.0f} | Runtime: {ctrl['runtime_hours']:.1f}h")
    
    if losers:
        print(f"\n📉 ALL LOSERS ({len(losers)}):")
        for ctrl in sorted(losers, key=lambda x: x['pnl_quote']):
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            print(f"  {dir_emoji} {ctrl['trading_pair']:12s} {ctrl['direction']:5s} | "
                  f"${ctrl['pnl_quote']:+7.2f} ({ctrl['pnl_pct']:+7.2f}%) | "
                  f"Vol: ${ctrl['volume']:>8,.0f} | Runtime: {ctrl['runtime_hours']:.1f}h")
    
    print("\n" + "=" * 80)
    print(f"💰 TOTAL P&L: ${total_pnl:+.2f}")
    print("=" * 80)
else:
    print(f"\n✅ No active bots - ready to deploy {CONFIG['bot_name']}")

## 3. Analyze Controllers - Decide What to Stop/Keep

In [ ]:
# Analyze each controller and decide action
controllers_to_stop = []
controllers_to_keep = []
stopped_controllers_inactive = []  # Controllers already stopped (not counted in capacity)
stop_reasons = {}

min_runtime_seconds = CONFIG['min_runtime_hours'] * 3600

print("🔍 Analyzing controllers...\n")

for ctrl in bot_controllers:
    ctrl_id = ctrl['controller_id']
    trading_pair = ctrl['trading_pair']
    direction = ctrl['direction']
    pnl_pct = ctrl['pnl_pct']
    runtime_hours = ctrl['runtime_hours']
    
    should_stop = False
    reason = None
    
    # Check if already manually stopped - DON'T COUNT THESE IN CAPACITY
    if ctrl['manual_kill_switch']:
        reason = "Already stopped (manual_kill_switch=True)"
        stopped_controllers_inactive.append(ctrl)
        print(f"  ⏸️  {trading_pair:12s} {direction:5s} - {reason}")
        continue
    
    # Check minimum runtime
    within_min_runtime = runtime_hours < CONFIG['min_runtime_hours']
    
    # Check signal alignment
    current_signal = signal_map.get(trading_pair)
    has_opposing_signal = False
    
    if current_signal:
        # Check if signal direction opposes controller direction
        if (direction == 'long' and current_signal['direction'] == 'short') or \
           (direction == 'short' and current_signal['direction'] == 'long'):
            has_opposing_signal = True
    
    # Decision logic
    if within_min_runtime:
        # Within minimum runtime - keep regardless of P&L or signal
        controllers_to_keep.append(ctrl)
        print(f"  ✅ {trading_pair:12s} {direction:5s} - Keep (runtime {runtime_hours:.1f}h < {CONFIG['min_runtime_hours']}h min) | P&L: ${ctrl['pnl_quote']:+.2f}")
    else:
        # Past minimum runtime - check stop conditions
        if CONFIG['stop_on_opposing_signal'] and has_opposing_signal:
            should_stop = True
            reason = f"Opposing signal (running {direction}, signal is {current_signal['direction']})"
        elif pnl_pct < CONFIG['min_pnl_pct_to_keep']:
            should_stop = True
            reason = f"Poor P&L ({pnl_pct:.2f}% < {CONFIG['min_pnl_pct_to_keep']}%)"
        else:
            controllers_to_keep.append(ctrl)
            signal_status = "✨ matching signal" if current_signal and current_signal['direction'] == direction else "no signal"
            print(f"  ✅ {trading_pair:12s} {direction:5s} - Keep ({signal_status}) | P&L: ${ctrl['pnl_quote']:+.2f} ({pnl_pct:+.2f}%)")
    
    if should_stop:
        controllers_to_stop.append(ctrl)
        stop_reasons[ctrl_id] = reason
        print(f"  🛑 {trading_pair:12s} {direction:5s} - STOP: {reason} | P&L: ${ctrl['pnl_quote']:+.2f}")

print(f"\n📊 Analysis Summary:")
print(f"  Active controllers: {len(controllers_to_keep)}")
print(f"  Controllers to stop: {len(controllers_to_stop)}")
print(f"  Already stopped (inactive): {len(stopped_controllers_inactive)}")

## 4. Stop Controllers

In [ ]:
# Stop controllers that should be stopped - MULTI-BOT SUPPORT
stopped_controllers = []

if controllers_to_stop and CONFIG['auto_stop_controllers']:
    print(f"🛑 Stopping {len(controllers_to_stop)} controllers...\n")
    
    async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
        for ctrl in controllers_to_stop:
            ctrl_id = ctrl['controller_id']
            bot_name = ctrl['bot_name']  # Use the bot_name from the controller
            reason = stop_reasons.get(ctrl_id, 'Unknown')
            
            try:
                # Update controller config to set manual_kill_switch = True
                await client.controllers.update_bot_controller_config(
                    bot_name=bot_name,  # Use the controller's specific bot
                    controller_name=ctrl_id,
                    config={'manual_kill_switch': True}
                )
                
                stopped_controllers.append(ctrl)
                print(f"  ✅ Stopped {ctrl['trading_pair']:12s} {ctrl['direction']:5s} (bot: {bot_name}) - {reason}")
                
            except Exception as e:
                print(f"  ❌ Failed to stop {ctrl['trading_pair']}: {e}")
    
    print(f"\n✅ Successfully stopped {len(stopped_controllers)} controllers")
elif controllers_to_stop:
    print(f"⚠️  Auto-stop is disabled. Would stop {len(controllers_to_stop)} controllers:")
    for ctrl in controllers_to_stop:
        print(f"  - {ctrl['trading_pair']} {ctrl['direction']}: {stop_reasons.get(ctrl['controller_id'])}")
else:
    print("ℹ️  No controllers to stop")

# Archive bots that have no active controllers
print("\n🔍 Checking for bots with all controllers stopped...\n")

# Group all controllers (active + stopped) by bot
all_controllers_by_bot = {}
for ctrl in bot_controllers:
    bot = ctrl['bot_name']
    if bot not in all_controllers_by_bot:
        all_controllers_by_bot[bot] = {'active': [], 'stopped': []}
    
    if ctrl['manual_kill_switch']:
        all_controllers_by_bot[bot]['stopped'].append(ctrl)
    else:
        all_controllers_by_bot[bot]['active'].append(ctrl)

bots_to_archive = []

for bot_name, controllers in all_controllers_by_bot.items():
    active_count = len(controllers['active'])
    stopped_count = len(controllers['stopped'])
    
    if active_count == 0 and stopped_count > 0:
        # All controllers are stopped - this bot should be archived
        bots_to_archive.append(bot_name)
        print(f"  🗂️  {bot_name}: All {stopped_count} controllers stopped - will archive")
    else:
        print(f"  ✅ {bot_name}: {active_count} active, {stopped_count} stopped - keeping")

if bots_to_archive and CONFIG['auto_stop_controllers']:
    print(f"\n📦 Archiving {len(bots_to_archive)} bot(s)...\n")
    
    async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
        for bot_name in bots_to_archive:
            try:
                await client.bot_orchestration.stop_bot(bot_name=bot_name)
                print(f"  ✅ Archived bot: {bot_name}")
            except Exception as e:
                print(f"  ❌ Failed to archive {bot_name}: {e}")
    
    print(f"\n✅ Successfully archived {len(bots_to_archive)} bot(s)")
elif bots_to_archive:
    print(f"\n⚠️  Would archive {len(bots_to_archive)} bot(s) (auto-stop disabled)")
else:
    print("\nℹ️  No bots to archive")

## 5. Determine New Controllers to Deploy

In [ ]:
# Helper function to calculate grid levels
async def calculate_grid_levels(trading_pair, connector_name, feature_name, direction='long'):
    features = await storage.get_features(
        feature_name=feature_name,
        trading_pair=trading_pair,
        connector_name=connector_name,
        limit=1
    )
    
    if not features:
        raise ValueError(f"No feature found for {trading_pair}")
    
    feat = features[0]
    price = feat.value['price']
    range_pct = feat.value['range_pct']
    multipliers = CONFIG['grid_range_multiplier'][direction]
    
    return {
        'current_price': price,
        'range_pct': range_pct,
        'start_price': price * (1 + multipliers['start_offset'] * range_pct),
        'end_price': price * (1 + multipliers['end_offset'] * range_pct),
        'limit_price': price * (1 + multipliers['limit_offset'] * range_pct),
        'feature_timestamp': feat.timestamp
    }

def generate_grid_config(trading_pair, side, signal_value, grid_levels):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    direction = "long" if side == 1 else "short"
    config_id = f"tf_{trading_pair.lower().replace('-', '_')}_{direction}_{timestamp}"

    return {
        'id': config_id,
        'controller_name': 'grid_strike',
        'controller_type': 'generic',
        'connector_name': CONFIG['connector_name'],
        'trading_pair': trading_pair,
        'side': side,
        'position_mode': CONFIG['position_mode'],
        'leverage': CONFIG['leverage'],
        'start_price': str(grid_levels['start_price']),
        'end_price': str(grid_levels['end_price']),
        'limit_price': str(grid_levels['limit_price']),
        'total_amount_quote': CONFIG['total_amount_quote'],
        'max_open_orders': CONFIG['max_open_orders'],
        'max_orders_per_batch': CONFIG['max_orders_per_batch'],
        'min_order_amount_quote': CONFIG['min_order_amount_quote'],
        'min_spread_between_orders': CONFIG['min_spread_between_orders'],
        'activation_bounds': CONFIG['activation_bounds'],
        'order_frequency': 5,
        'keep_position': False,
        'triple_barrier_config': {
            'open_order_type': 3,
            'take_profit': CONFIG['take_profit'],
            'take_profit_order_type': 3
        },
        'signal_value': signal_value,
    }

# Determine how many controllers we can deploy
current_active_controllers = len(controllers_to_keep)
max_new_controllers = CONFIG['max_controllers_per_instance'] - current_active_controllers

print(f"📊 Controller Capacity:")
print(f"  Controllers to keep: {len(controllers_to_keep)}")
print(f"  Controllers to stop: {len(stopped_controllers)}")
print(f"  Available slots: {max_new_controllers}\n")

# Get markets that are ACTUALLY staying active (excluding stopped ones)
running_markets = {ctrl['trading_pair']: ctrl['direction'] for ctrl in controllers_to_keep}
stopped_markets = {ctrl['trading_pair']: ctrl['direction'] for ctrl in stopped_controllers}

# Identify new opportunities
new_configs = []
new_long_signals = [sig for sig in long_signals[:CONFIG['top_n_per_side']]]
new_short_signals = [sig for sig in short_signals[:CONFIG['top_n_per_side']]]

print("🔍 Analyzing opportunities for new controllers...\n")

# Process long signals
for signal in new_long_signals:
    if len(new_configs) >= max_new_controllers:
        break
    
    trading_pair = signal.trading_pair
    
    # Skip if already running in same direction (keeping it)
    if trading_pair in running_markets and running_markets[trading_pair] == 'long':
        print(f"  ⏭️  {trading_pair:12s} LONG  - Already running, keeping")
        continue
    
    # Skip if just stopped - don't redeploy immediately
    if trading_pair in stopped_markets and stopped_markets[trading_pair] == 'long':
        print(f"  ⛔ {trading_pair:12s} LONG  - Just stopped, skipping")
        continue
    
    # Generate config for new controller
    try:
        grid_levels = await calculate_grid_levels(
            trading_pair, CONFIG['connector_name'], CONFIG['feature_name'], 'long'
        )
        config = generate_grid_config(trading_pair, 1, signal.value, grid_levels)
        new_configs.append(config)
        print(f"  🆕 {trading_pair:12s} LONG  - New opportunity | Signal: {signal.value:+.3f}")
    except Exception as e:
        print(f"  ❌ {trading_pair} LONG: {e}")

# Process short signals
for signal in new_short_signals:
    if len(new_configs) >= max_new_controllers:
        break
    
    trading_pair = signal.trading_pair
    
    # Skip if already running in same direction (keeping it)
    if trading_pair in running_markets and running_markets[trading_pair] == 'short':
        print(f"  ⏭️  {trading_pair:12s} SHORT - Already running, keeping")
        continue
    
    # Skip if just stopped - don't redeploy immediately
    if trading_pair in stopped_markets and stopped_markets[trading_pair] == 'short':
        print(f"  ⛔ {trading_pair:12s} SHORT - Just stopped, skipping")
        continue
    
    # Generate config for new controller
    try:
        grid_levels = await calculate_grid_levels(
            trading_pair, CONFIG['connector_name'], CONFIG['feature_name'], 'short'
        )
        config = generate_grid_config(trading_pair, 2, signal.value, grid_levels)
        new_configs.append(config)
        print(f"  🆕 {trading_pair:12s} SHORT - New opportunity | Signal: {signal.value:+.3f}")
    except Exception as e:
        print(f"  ❌ {trading_pair} SHORT: {e}")

print(f"\n✅ Identified {len(new_configs)} new controllers to deploy")

if stopped_controllers and not new_configs:
    print(f"\n⚠️  Note: Stopped {len(stopped_controllers)} controllers but no new opportunities available")
    print("   Consider expanding signal search or adjusting min intensity thresholds")

## 6. Visualize New Opportunities

In [ ]:
# Visualize candles with EMAs and grid levels for new opportunities BEFORE deploying
if new_configs:
    print(f"📊 Generating charts for {len(new_configs)} new opportunities...\n")

    # Load candles from cache
    from core.data_sources.clob import CLOBDataSource
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    clob = CLOBDataSource()
    clob.load_candles_cache()

    # Create a dict for quick lookup
    candles_dict = {}
    for key, value in clob.candles_cache.items():
        if key[-1] == '1m' and key[0] == CONFIG['connector_name']:
            candles_dict[key[1]] = value  # key[1] is trading_pair

    # Visualize each new opportunity
    for config in new_configs:
        trading_pair = config['trading_pair']
        direction = "LONG" if config['side'] == 1 else "SHORT"

        # Get candles for this pair
        if trading_pair not in candles_dict:
            print(f"  ⚠️  No candles found for {trading_pair}")
            continue

        candles = candles_dict[trading_pair]

        # Filter to last 7 days using df property
        seven_days_ago = datetime.now(timezone.utc) - timedelta(days=7)
        candles.data = candles.data[candles.data['timestamp'] >= seven_days_ago.timestamp()].copy()

        # Create figure with candlestick
        fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

        # Add candlesticks
        fig.add_trace(
            candles.candles_trace()
        )

        # Add grid levels as horizontal lines
        grid_levels = {
            'start_price': float(config['start_price']),
            'end_price': float(config['end_price']),
            'limit_price': float(config['limit_price']),
            'current_price': df['close'].iloc[-1]
        }

        # Color scheme based on direction
        if direction == "LONG":
            start_color = '#10AC84'  # Green for long entry zone
            end_color = '#48DBFB'    # Light blue for long target
            limit_color = '#FF6B6B'  # Red for long stop
        else:
            start_color = '#FF6B6B'  # Red for short entry zone
            end_color = '#10AC84'    # Green for short target
            limit_color = '#48DBFB'  # Light blue for short stop

        # Add grid boundary lines
        fig.add_hline(
            y=grid_levels['start_price'],
            line_dash="dash",
            line_color=start_color,
            line_width=2,
            annotation_text=f"Start: ${grid_levels['start_price']:.6f}",
            annotation_position="right"
        )

        fig.add_hline(
            y=grid_levels['end_price'],
            line_dash="dash",
            line_color=end_color,
            line_width=2,
            annotation_text=f"End: ${grid_levels['end_price']:.6f}",
            annotation_position="right"
        )

        fig.add_hline(
            y=grid_levels['limit_price'],
            line_dash="dot",
            line_color=limit_color,
            line_width=2,
            annotation_text=f"Limit: ${grid_levels['limit_price']:.6f}",
            annotation_position="right"
        )

        # Update layout
        dir_emoji = "🟢" if direction == "LONG" else "🔴"
        signal_value = config.get('signal_value', 0)

        fig.update_layout(
            title=f"{dir_emoji} {trading_pair} - {direction} Grid (Signal: {signal_value:+.3f})<br><sub>Last 7 Days </sub>",
            xaxis_title="Time",
            yaxis_title="Price (USDT)",
            height=600,
            width=1400,
            hovermode='x unified',
            template='plotly_dark',
            xaxis_rangeslider_visible=False,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        fig.show()
        print(f"  ✅ {trading_pair} {direction} chart generated\n")

    print(f"✅ Generated {len(new_configs)} charts")
else:
    print("ℹ️  No new opportunities to visualize")

## 7. Deploy New Controllers

In [ ]:
# Deploy new controllers - MULTI-BOT SUPPORT WITH LIMITS
deployed_configs = []

if new_configs and CONFIG['auto_deploy']:
    # Check bot limit before deploying
    active_bots_count = len([bot for bot in all_controllers_by_bot.keys() if bot not in bots_to_archive])
    
    if active_bots_count >= CONFIG['max_active_bots']:
        print(f"⚠️  Maximum bot limit reached ({active_bots_count}/{CONFIG['max_active_bots']})")
        print(f"   Cannot deploy new bot until existing bots are archived")
        print(f"   Currently active bots:")
        for bot_name in all_controllers_by_bot.keys():
            if bot_name not in bots_to_archive:
                active = len(all_controllers_by_bot[bot_name]['active'])
                stopped = len(all_controllers_by_bot[bot_name]['stopped'])
                print(f"     - {bot_name}: {active} active, {stopped} stopped")
    else:
        print(f"🚀 Deploying {len(new_configs)} new controllers...")
        print(f"   Current bots: {active_bots_count}/{CONFIG['max_active_bots']}\n")
        
        async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
            # Create controller configs
            for config in new_configs:
                try:
                    api_config = {k: v for k, v in config.items() if k not in ['signal_value']}
                    
                    await client.controllers.create_or_update_controller_config(
                        config_name=config['id'],
                        config=api_config
                    )
                    
                    deployed_configs.append(config)
                    direction = "LONG" if config['side'] == 1 else "SHORT"
                    print(f"  ✅ Created config: {config['trading_pair']:12s} {direction}")
                    
                except Exception as e:
                    print(f"  ❌ Failed to create config for {config['trading_pair']}: {e}")
            
            # Deploy bot with configs
            if deployed_configs:
                try:
                    await client.bot_orchestration.deploy_v2_controllers(
                        instance_name=CONFIG['bot_name'],  # Use bot_name directly
                        credentials_profile=CONFIG['credentials_profile'],
                        controllers_config=[c['id'] for c in deployed_configs]
                    )
                    print(f"\n✅ Successfully deployed bot '{CONFIG['bot_name']}' with {len(deployed_configs)} controllers")
                        
                except Exception as e:
                    print(f"❌ Failed to deploy bot: {e}")
                    
elif new_configs:
    print(f"⚠️  Auto-deploy disabled. Would deploy {len(new_configs)} controllers")
else:
    print("ℹ️  No new controllers to deploy")

## 8. Send Telegram Report

In [ ]:
# Final Summary Report via Telegram - CONSOLIDATED SINGLE MESSAGE
if CONFIG['send_telegram_notifications']:
    telegram_message = f"""<b>🎉 Bot Orchestration Complete</b>
━━━━━━━━━━━━━━━━━━━━━━━━━━━━

"""

    # SECTION 1: Current Active Bots Status
    if bot_controllers:
        # Get active controllers only (not stopped ones)
        active_controllers = [c for c in bot_controllers if not c['manual_kill_switch']]
        total_pnl = sum(ctrl['pnl_quote'] for ctrl in active_controllers)
        total_volume = sum(ctrl['volume'] for ctrl in active_controllers)
        winners = [c for c in active_controllers if c['pnl_quote'] > 0]
        losers = [c for c in active_controllers if c['pnl_quote'] < 0]
        
        # Group controllers by bot
        controllers_by_bot = {}
        for ctrl in active_controllers:
            bot = ctrl['bot_name']
            if bot not in controllers_by_bot:
                controllers_by_bot[bot] = []
            controllers_by_bot[bot].append(ctrl)
        
        telegram_message += f"""<b>📊 Active Bots ({len(controllers_by_bot)})</b>

"""
        for bot_name, controllers in controllers_by_bot.items():
            bot_pnl = sum(c['pnl_quote'] for c in controllers)
            bot_runtime = controllers[0]['runtime_hours'] if controllers else 0
            telegram_message += f"""<b>📦 {bot_name}</b>
├ Runtime: {bot_runtime:.1f}h
├ Controllers: {len(controllers)} active
└ P&amp;L: <code>${bot_pnl:+.2f}</code>

"""
        
        telegram_message += f"""<b>💰 Total Performance</b>
├ P&amp;L: <code>${total_pnl:+.2f}</code>
├ Volume: ${total_volume:,.0f}
└ W/L: {len(winners)}/{len(losers)}

"""
    else:
        telegram_message += "<b>📊 Active Bots</b>\nNo active bots before this run.\n\n"

    # SECTION 2: Stopped Controllers
    if stopped_controllers:
        telegram_message += f"<b>🛑 Controllers Stopped ({len(stopped_controllers)})</b>\n"
        for ctrl in stopped_controllers:
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            reason = stop_reasons.get(ctrl['controller_id'], 'Unknown')
            reason_safe = reason.replace('<', '&lt;').replace('>', '&gt;').replace('&', '&amp;')
            telegram_message += f"{dir_emoji} <code>{ctrl['trading_pair']:10s}</code> ${ctrl['pnl_quote']:+.2f} - {reason_safe}\n"
        telegram_message += "\n"

    # SECTION 3: Bots Archived
    if bots_to_archive:
        telegram_message += f"<b>🗂️  Bots Archived ({len(bots_to_archive)})</b>\n"
        for bot_name in bots_to_archive:
            telegram_message += f"• <code>{bot_name}</code>\n"
        telegram_message += "\n"

    # SECTION 4: New Bot Deployed
    if deployed_configs:
        telegram_message += f"""<b>🚀 New Bot Deployed</b>

<b>Controllers:</b> {len(deployed_configs)}
<b>Capital:</b> ${float(CONFIG['total_amount_quote']) * len(deployed_configs):,.0f}

"""
        # Group by direction
        longs = [c for c in deployed_configs if c['side'] == 1]
        shorts = [c for c in deployed_configs if c['side'] == 2]
        
        if longs:
            telegram_message += f"<b>🟢 Long ({len(longs)})</b>\n"
            for config in longs:
                telegram_message += f"• <code>{config['trading_pair']:10s}</code> Signal: {config['signal_value']:+.3f}\n"
            telegram_message += "\n"
        
        if shorts:
            telegram_message += f"<b>🔴 Short ({len(shorts)})</b>\n"
            for config in shorts:
                telegram_message += f"• <code>{config['trading_pair']:10s}</code> Signal: {config['signal_value']:+.3f}\n"
            telegram_message += "\n"

    # SECTION 5: Management Summary
    telegram_message += f"""<b>⚙️ Management Rules</b>
├ Min Runtime: <code>{CONFIG['min_runtime_hours']}h</code>
├ Min P&amp;L: <code>{CONFIG['min_pnl_pct_to_keep']}%</code>
└ Stop on Signal Change: <code>{CONFIG['stop_on_opposing_signal']}</code>

<i>📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}</i>"""

    notification = NotificationMessage(
        title="🎉 Orchestration Complete",
        message=telegram_message,
        level="info",
    )

    print("📤 Sending final report via Telegram...\n")
    
    try:
        results = await notification_manager.send_notification(notification)
        
        for service, success in results.items():
            status = "✅ Sent" if success else "❌ Failed"
            print(f"  {service.capitalize()}: {status}")
        
        if results.get('telegram'):
            print("\n🎉 Final report sent to Telegram!")
            
    except Exception as e:
        print(f"❌ Error sending notification: {e}")
else:
    print("ℹ️  Telegram notifications disabled")

## 9. Summary

In [ ]:
print("\n" + "="*80)
print("🎉 SMART BOT ORCHESTRATION COMPLETE")
print("="*80)
print(f"\n📊 Final State:")
print(f"  Bot Name: {CONFIG['bot_name']}")
print(f"  Active Controllers: {len(controllers_to_keep) + len(deployed_configs)}")
print(f"  Stopped Controllers: {len(stopped_controllers)}")
print(f"  New Controllers Deployed: {len(deployed_configs)}")

if controllers_to_keep:
    total_pnl = sum(ctrl['pnl_quote'] for ctrl in controllers_to_keep)
    print(f"\n💰 Active Controllers P&L: ${total_pnl:+.2f}")

if deployed_configs:
    total_capital = float(CONFIG['total_amount_quote']) * len(deployed_configs)
    print(f"💵 Newly Deployed Capital: ${total_capital:,.0f}")

print("\n" + "="*80)